# Displaying North East South West for 1 time step with leaflet

In [4]:
import sys
sys.path.append("../helpers/")
import helpers
from src.datacube.dq_database.db_view import DQDataBaseView

from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles, TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle,  GeoJSON, WidgetControl, DrawControl, LayerGroup, FullScreenControl, interactive)

from ipywidgets import IntSlider, ColorPicker, Button, link, Label, GridspecLayout, Layout, HBox, VBox, jslink
import ipywidgets as widgets
import datetime
dqbv = DQDataBaseView()

def on_button_clicked(b):
    h = helpers.Helpers(out)
    h.color_map_nesw(product.value, subproduct.value, north.value,east.value, south.value, west.value,
                                                  date, hour)

m = Map(center=(1, -5), zoom=2, layout=dict(width='400px', height='300px'))

dc = DrawControl()
m.add_control(dc)

helpers.Helpers.prepare_map(dc, m)

w = helpers.Widgets()
product = w.get_all_products()
subproduct = w.subproduct()
north = w.get_point(38.0, 'North: ')
east = w.get_point(53.0, 'East: ')
south = w.get_point((-36), 'South: ')
west = w.get_point((-19), 'West: ')
date = w.get_date(datetime.datetime(2018,1,1), 'Date: ')
hour = w.get_hour('00', 'Hour: ')
button = w.set_up_button(on_button_clicked)
w.display_widgets(product, subproduct, north, east, south, west, date, hour, button, m)

label = Label()
helpers.Helpers.mouse_interaction(m, label)

def update_north(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (north, 'value'))

def update_east(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (east, 'value'))

def update_south(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (south, 'value'))

def update_west(x):
    w = helpers.Helpers.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (west, 'value'))    
    

def handle_draw(self, action, geo_json):
    w = helpers.Widgets()
    s = geo_json.get('geometry','type')
    t = s.get('type')

    h = helpers.Helpers(out)
    if t == 'Point':
        north, east, south, west = h.get_coords_point(action, geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

    if t == 'Polygon':
        north, east, south, west = h.get_coords_polygon(action, geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

def update(*args):
    subproduct.options = dqbv.get_subproducts_from_product_name(product.value)
update()
product.observe(update)

out = w.display_output()

dc.on_draw(handle_draw)


Label(value='')

Output()